<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/OpenMM/OpenMM_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openmm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 16.8 MB/s eta 0:00:00


In [2]:
!python -m openmm.testInstallation


OpenMM Version: 8.2
Git Revision: 53770948682c40bd460b39830d4e0f0fd3a4b868

There are 2 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.30854e-06

All differences are within tolerance.


In [12]:

# Define filenames
fixed_pdb = "/content/functionalized_graphene.pdb"  # Rewritten PDB
mol2_file = "graphene.mol2"  # GAFF parameters with atom types
frcmod_file = "graphene.frcmod"  # Additional parameters
tleap_input = "tleap_graphene.in"  # tleap input script
prmtop_file = "graphene.prmtop"  # Amber topology file
inpcrd_file = "graphene.inpcrd"  # Amber coordinate file

# Step 2: Run Antechamber to generate GAFF parameters
print("Running antechamber...")
os.system(f"antechamber -i {fixed_pdb} -fi pdb -o {mol2_file} -fo mol2 -c bcc -s 2 -nc 0")

# Step 3: Check for missing parameters using parmchk2
print("Running parmchk2...")
os.system(f"parmchk2 -i {mol2_file} -f mol2 -o {frcmod_file}")

# Step 4: Create tleap input file
tleap_content = f"""
source leaprc.gaff

# Load parameters and molecule
loadAmberParams {frcmod_file}
grf = loadMol2 {mol2_file}

# Save outputs
saveAmberParm grf {prmtop_file} {inpcrd_file}
savePdb grf graphene_leap_out.pdb

quit
"""
with open(tleap_input, 'w') as tleap_file:
    tleap_file.write(tleap_content)
print(f"tleap input file saved as {tleap_input}.")

# Step 5: Run tleap
print("Running tleap...")
os.system(f"tleap -f {tleap_input}")

Running antechamber...
Running parmchk2...
tleap input file saved as tleap_graphene.in.
Running tleap...


32512

In [10]:
!conda install -c conda-forge ambertools -y

/bin/bash: line 1: conda: command not found


In [8]:
from openmm.app import PDBFile, ForceField, Modeller, Simulation, PME
from openmm import VerletIntegrator
from openmm.unit import nanometer, picoseconds
import parmed as pmd

# Load initial structure
pdb = PDBFile('/content/functionalized_graphene.pdb')

# Include force fields (ensure 'mercury2.xml' defines Hg²⁺ parameters)
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

# Create modeller and add missing hydrogens
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)

# Add solvent with Hg²⁺ and Cl⁻ ions for neutrality, TIP3P water, and 1 nm padding
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer,
                    positiveIon='Hg2+', negativeIon='Cl-')

# Create system, integrator, and simulation
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
integrator = VerletIntegrator(0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)

# Set positions and minimize energy
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)

# Retrieve minimized positions
state = simulation.context.getState(getPositions=True)
positions = state.getPositions()

# Optionally save final structure as PDB
with open('output.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, positions, f)

# Convert OpenMM topology and positions to a ParmEd Structure
structure = pmd.openmm.load_topology(simulation.topology, positions)

# Save as LAMMPS data file
structure.save('system.lmp', format='lammps-data')

ValueError: No template found for residue 1 (GGG).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

In [9]:
from openmm.app import PDBFile, ForceField, Modeller, Simulation, PME
from openmm import VerletIntegrator
from openmm.unit import nanometer, picoseconds
import parmed as pmd

# Load initial structure
pdb = PDBFile('/content/functionalized_graphene.pdb')

# Use force fields without mercury2.xml since it's not available
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

# Create modeller without adding hydrogens to avoid template issues
modeller = Modeller(pdb.topology, pdb.positions)
# Skip addHydrogens because of unknown residue templates:
# modeller.addHydrogens(forcefield)

# Add solvent with allowed ions (Na+ and Cl-) for neutrality
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer,
                    positiveIon='Na+', negativeIon='Cl-')

# Create system, integrator, and simulation
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
integrator = VerletIntegrator(0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)

# Set positions and minimize energy
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)

# Retrieve minimized positions
state = simulation.context.getState(getPositions=True)
positions = state.getPositions()

# Optionally save final structure as PDB
with open('output.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, positions, f)

# Convert OpenMM topology and positions to a ParmEd Structure
structure = pmd.openmm.load_topology(simulation.topology, positions)

# Save as LAMMPS data file
structure.save('system.lmp', format='lammps-data')

ValueError: No template found for residue 1 (GGG).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

In [13]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np

# Define parameters for the graphene sheet
def generate_graphene_sheet(nx, ny, bond_length):
    """
    Generates a graphene sheet with nx x ny atoms.
    bond_length: The length of C-C bonds in nanometers.
    Returns: topology and positions.
    """
    positions = []
    topology = Topology()
    chain = topology.addChain()
    residue = topology.addResidue("GRP", chain)

    for i in range(nx):
        for j in range(ny):
            x = i * bond_length
            y = j * bond_length * np.sqrt(3) / 2
            if j % 2 == 1:
                x += bond_length / 2
            atom = topology.addAtom("C", element.carbon, residue)
            positions.append(Vec3(x, y, 0.0) * nanometer)

    return topology, positions

# Graphene dimensions
nx, ny = 10, 10  # Number of atoms in x and y
bond_length = 0.142  # Bond length in nanometers

# Generate the graphene sheet
graphene_topology, graphene_positions = generate_graphene_sheet(nx, ny, bond_length)

# Create a system and add a force field
forcefield = ForceField("amber14-all.xml", "tip3p.xml")

# Add water molecules
modeller = Modeller(graphene_topology, graphene_positions)
modeller.addSolvent(forcefield, model='tip3p', boxSize=Vec3(3.0, 3.0, 3.0)*nanometer)

# Create the system
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
                                 nonbondedCutoff=1.0*nanometer, constraints=HBonds)

# Integrator
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

# Simulation context
platform = Platform.getPlatformByName("CPU")
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# Output PDB file with water
with open("graphene_with_water.pdb", "w") as f:
    PDBFile.writeFile(modeller.topology, modeller.positions, f)

print("Graphene with water molecules has been generated and written to 'graphene_with_water.pdb'.")


ValueError: No template found for residue 1 (GRP).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

ValueError: No template found for residue 1 (GRP).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

In [5]:
!pip install parmed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for parmed: filename=ParmEd-4.3.0-cp311-cp311-linux_x86_64.whl size=19471828 sha256=1d224f2d874e0848b3b180850a8501a3f001a68675a2b9423c2ecfaa64fd8ee7
  Stored in directory: /root/.cache/pip/wheels/5d/5f/0a/e88e403aba1e99f00bdf228e123bbf9543a881e568079fc065
Successfully built parmed


In [3]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

#new stuff

In [15]:
!pip install -q condacolab
import condacolab
condacolab.install()  # restarts runtime; rerun from here after restart

!mamba install -c conda-forge ambertools -y

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...

Looking for: ['ambertools']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64  ⣾  
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64   1%
conda-forge/noarch     1%[+] 0.4s
conda-forge/linux-64   3%
conda-forge/noarch     6%[+] 0.5s
conda-forge/linux-64   8%
conda-forge/noarch    27%[+] 0.6s
conda-forge/linux-64  16%
conda-forge/noarch    46%[+] 0.7s
conda-forge/linux-64  19%
conda-forge/noarch    55%[+] 0.8s
conda-forge/linux-64  22%
conda-forge/noarch    59%[+] 0.9s
conda-forge/linux-64  26%
conda-forge/noarch    66%[+] 1.0s
conda-forge/linux-64  28%
conda-forge/noarch    72%[+] 1.1s
conda-forge/linux-64  32%
conda-forge/noarch    81%[+] 1.2s
conda-forge/linux

In [1]:
!git clone https://github.com/Iourarum/GOPY.git
%cd GOPY
!python GOPY.py generate_PG 10 10 graphene.pdb
!python GOPY.py generate_GO graphene.pdb 2 4 8 functionalized.pdb

Cloning into 'GOPY'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 197 (delta 39), reused 18 (delta 18), pack-reused 150 (from 1)
Receiving objects: 100% (197/197), 9.50 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/GOPY
done.
GOPY.py
generate_GO
graphene.pdb
2
4
8
functionalized.pdb
Left to add:  cooh:  2 epoxy:  4 hydroxyl:  8
Left to add:  cooh:  2 epoxy:  3 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  3 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  2 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  2 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  1 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  6
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  5
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  4
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  3
Left to add:  cooh:  0 epoxy:  0

In [2]:
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/E1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/H1A.lib
!mkdir -p FRCMOD
!wget -P FRCMOD https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/FRCMOD/GO.frcmod

--2025-01-21 05:08:47--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1063 (1.0K) [text/plain]
Saving to: ‘GGG.lib’

GGG.lib             100%[===================>]   1.04K  --.-KB/s    in 0s      

2025-01-21 05:08:47 (35.7 MB/s) - ‘GGG.lib’ saved [1063/1063]

--2025-01-21 05:08:47--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1627 (1.6K) [text/plain]
Saving to: ‘C1A.lib’

C1

In [7]:
%%bash
cd /content/GOPY
cat > leap.in <<EOF
source leaprc.DNA.bsc1
loadoff GGG.lib
loadoff C1A.lib
loadoff E1A.lib
loadoff H1A.lib
loadamberparams FRCMOD/GO.frcmod
mol = loadpdb functionalized.pdb
bondbydistance mol
saveamberparm mol mol.prmtop mol.inpcrd
quit
EOF
tleap -f leap.in


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./leap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.DNA.bsc1
----- Source of /usr/local/dat/leap/cmd/leaprc.DNA.bsc1 done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading library: /usr/local/dat/leap/lib/parmBSC1.lib
Loading parameters: /usr/local/dat/leap/parm/frcmod.parmbsc1
Reading force field modification type file (frcmod)
Reading title:
Parmbsc1 force-field for DNA
Loading library: ./GGG.lib
Loading library: ./C1A.lib
Loading library: ./E1A.lib
Loading library: ./H1A.lib
Loading parameters: ./FRCMOD/GO.frcmod
Reading force field modification type file (frcmod)
Reading title:
Remark line goes here


In [8]:
!pip install openmm

  Using cached OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (844 bytes)
Using cached OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (12.3 MB)


In [16]:
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PME, PDBFile, HBonds, CutoffNonPeriodic
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import *
import parmed as pmd

# Load the Amber topology and coordinates
prmtop = AmberPrmtopFile('/content/GOPY/mol.prmtop')
inpcrd = AmberInpcrdFile('/content/GOPY/mol.inpcrd')


system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic,
                             nonbondedCutoff=1.0*nanometer,
                             constraints=HBonds)


# Define the integrator
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

# Select platform if necessary (e.g., CPU)
platform = Platform.getPlatformByName("CPU")

# Create the simulation object
simulation = Simulation(prmtop.topology, system, integrator, platform)

# Set initial positions from Amber coordinates
simulation.context.setPositions(inpcrd.positions)

# Minimize energy
simulation.minimizeEnergy()

# Optionally, write out the structure with water to a PDB file for inspection
state = simulation.context.getState(getPositions=True)
with open("functionalized_graphene_with_water.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, state.getPositions(), f)

print("Simulation setup complete. The structure has been minimized and written to 'functionalized_graphene_with_water.pdb'.")


Simulation setup complete. The structure has been minimized and written to 'functionalized_graphene_with_water.pdb'.


In [17]:
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PME, PDBFile, HBonds, Modeller, ForceField
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import *

# Load Amber topology and coordinates
prmtop = AmberPrmtopFile('/content/GOPY/mol.prmtop')
inpcrd = AmberInpcrdFile('/content/GOPY/mol.inpcrd')

# Extract initial topology and positions
topology = prmtop.topology
positions = inpcrd.positions

# Select force fields for solute and water/ions
forcefield = ForceField('amber14-all.xml', 'tip3p.xml')

# Use Modeller to add solvent and ions
modeller = Modeller(topology, positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer, ionicStrength=0.1*molar)

# Create system using PME now that we have a periodic box
system = forcefield.createSystem(
    modeller.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometer,
    constraints=HBonds
)

# Define integrator and select platform
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
platform = Platform.getPlatformByName("CPU")

# Initialize simulation with updated topology
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# (Optional) Run a short simulation
simulation.step(1000)  # adjust steps as needed

# Save the minimized structure to a PDB file
state = simulation.context.getState(getPositions=True)
with open("functionalized_graphene_with_water.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, state.getPositions(), f)

print("Simulation complete. Structure minimized with water and ions added.")

ValueError: No template found for residue 1 (GGG).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template